In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import codecs
from bs4 import BeautifulSoup
import time
import json
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import pairwise_distances
import pymorphy2
from multiprocessing.dummy import Lock
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer

In [9]:
morph = pymorphy2.MorphAnalyzer()

In [10]:
stemmerR = SnowballStemmer('russian')
stemmerE = SnowballStemmer('english')

In [11]:
def inp_for_titles(line):
    data = line.split('\t', 1)
    doc_id = int(data[0])
    if len(data[1]) == 1:
        title = ''
    else:
        title = data[1]
    doc_to_title[doc_id] = title[:-1]

In [12]:
doc_to_title = {}
with codecs.open('docs_titles.tsv', 'r', 'utf8') as f:
    next(f)
    with ThreadPool(10) as pool:
        pool.map(inp_for_titles, f)
print(len(doc_to_title))

28026


In [13]:
doc_to_title

{15731: 'ВАЗ 21213 | Замена подшипников ступицы | Нива',
 14829: 'Ваз 2107 оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
 15764: 'Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена, замена, тюнинг.',
 17669: 'Классика 21010 - 21074',
 14852: 'Ступица Нива — замена подшипника своими руками',
 15458: 'ВАЗ 2110',
 14899: 'Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123',
 16879: 'Купить Подшипники и ступицы FAG (Страница 23)',
 16310: 'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года',
 15440: 'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти',
 16242: 'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
 16383: 'Ступицы - OLX.ua - страница 80',
 15580: 'маааленькая проблемка — бортжурнал Автокам 2160 ╬ 1994 года на DRIVE2',
 16840: 'Разгруженные полуоси для Нива (24 шлица 765 мм)',
 17519: 'Прошивки для нива м7.9.7 скачать - Фа

In [14]:
stop_words = set(stopwords.words(["russian", "english"]))

In [21]:
lock = Lock()

In [23]:
def Parser_for_titles(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [stemmerR.stem(stemmerE.stem(word)) for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [24]:
vec = {}
with ThreadPool(10) as pool:
    pool.map(Parser_for_titles, list(range(1,28027)))

In [25]:
vec

{1: 'м б аншин центр репродукц и генетик фертимед г москв',
 2: 'перевод кив кошельк',
 3: 'проект патрул времен реабилитац духовн существ просмотр тем деньг и экономик обм и финанс деньг без процент',
 4: 'блог клуб преподаван в начальн класс порта профессиональн образован чеченск республик',
 702: 'медицинск порта узбекиста med uz дерматовенеролог рус',
 703: 'уста',
 704: 'как состав бизнес план карьер и сво бизнес',
 705: 'вариац на тем подкормк томат стр 1 томат в теплиц форум',
 706: 'зубн врач и врач стоматолог в чем разниц особен професс',
 1403: 'где в казан можн куп медилис ципер наш справк в город казан татарста',
 1404: 'первичн бесплод причин диагностик профилактик',
 1405: 'стен вконтакт',
 1406: 'экстернат больш не будет',
 1407: 'мыш 2 штук рабоч оптическ цен за 2 мыш куп в красноярск состоян б у клавиатур и мыш на интернет аукцион au ru',
 707: 'как уменьш холестерин в кров в домашн услов народн средств здоров питан и фитнес golbi',
 708: 'современ сатирическ рома лите

In [27]:
with open('titles_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

Застемил заголовки - уже прекрасно!

## Мутим фичи

In [1]:
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-macosx_10_12_x86_64.whl size=324287 sha256=1aca536a5f62e01a9433b11746a610136aa323a703f84c1f45af47de11095c96
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 2.2 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.0.1-py3-none-any.whl size=108249 sha256=54b93ee534545cb3657fe4c27116c6ea53bd50ab3c37028568418f11c50e2d07
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/34/3d/14/f19c01a19c9201cdb6a76b049904d5226912569be919ad1eae
Successfully built smart-open
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
from gensim.models import FastText

from sklearn.metrics.pairwise import cosine_similarity

Взяли заголовки

In [23]:
vec = {}

In [24]:
with open('titles_stemming.json', 'r', encoding='utf8') as file:
    vec = json.load(file)

In [25]:
sentences = []
for value in vec.values():
    sentences.append(value)

In [26]:
model_ted = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=1)

In [32]:
len(sentences)

28026

In [27]:
first = sentences[0]
first

'м б аншин центр репродукц и генетик фертимед г москв'

In [47]:
vec['15731']

'ваз 21213 зам подшипник ступиц нив'

In [28]:
embedding = model_ted.wv[first]

In [29]:
embedding

array([ 2.88571900e-04,  4.43181460e-04, -1.82474560e-05, -4.42146411e-04,
       -1.81859723e-04, -2.62079237e-04,  5.00643800e-04, -3.25563626e-04,
       -1.41036307e-05,  8.73165380e-04,  5.52282610e-04,  4.32018976e-04,
       -4.86722420e-04,  6.30522438e-04,  3.91866983e-04,  2.32932245e-04,
        4.15771414e-04, -1.48380554e-04,  7.69641483e-04, -7.16354290e-04,
       -1.57203060e-04, -5.77202532e-04,  2.60940171e-04, -1.01758778e-04,
       -1.16324506e-03, -5.80957785e-05, -3.91297828e-04,  7.26800004e-04,
       -3.82551429e-04, -7.58286391e-04, -1.36312970e-04,  4.67929058e-04,
       -8.33523336e-06, -1.26375118e-04,  1.04714556e-04, -2.23592491e-04,
       -3.30124138e-04,  5.31728801e-05, -1.98038106e-04,  1.29094449e-04,
       -1.36508694e-04, -9.80084296e-04,  3.56185046e-04,  3.38119899e-05,
       -2.72152189e-04, -4.29621141e-04, -4.46054211e-04,  2.83930531e-05,
        3.55043216e-04, -3.99042998e-04,  2.02465715e-04,  2.09369959e-04,
        6.71668386e-04,  

In [60]:
class Import():
    def __init__(self):
        pass
    
    def import_from_file(self, file):
        '''
        df = pd.read_csv(file)
        groups = df.groupby('group_id')
        if 'target' in df.columns:
            X = np.empty(shape=(df.shape[0]), dtype=np.float)
            y = np.empty(shape=(df.shape[0]), dtype=bool)
            group_ids = np.empty(shape=(df.shape[0], ), dtype=int)
            i = 0
            print(groups.groups.items())
            return X, y, group_ids
        '''
        doc_to_title = {}
        with open('/Users/egor/Desktop/Техносфера/MachineLearning/project/docs_titles.tsv') as f:
            for num_line, line in enumerate(f):
                if num_line == 0:
                    continue
                data = line.strip().split('\t', 1)
                doc_id = int(data[0])
                if len(data) == 1:
                    title = ''
                else:
                    title = data[1]
                doc_to_title[doc_id] = title


        train_data = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/train_groups.csv')
        traingroups_titledata = {}
        for i in range(len(train_data)):
            new_doc = train_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            target = new_doc['target']
            title = doc_to_title[doc_id]
            if doc_group not in traingroups_titledata:
                traingroups_titledata[doc_group] = []
            traingroups_titledata[doc_group].append((doc_id, title, target))
        
        # Fasttext embeddings
        y_train = []
        X_train = []
        X_embed = []
        groups_train = []
        for new_group in traingroups_titledata:
            docs = traingroups_titledata[new_group]
            for k, (doc_id, title, target_id) in enumerate(docs):
                y_train.append(target_id)
                groups_train.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j, target_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_train.append(sorted(all_dist, reverse=True)[0:25]    )
                # embeddings
                X_embed.append(model_ted.wv[vec[str(doc_id)]])
        X_train = np.array(X_train)
        X_embed = np.array(X_embed)
        X_train = np.concatenate((X_train, X_embed), axis=1)
        y_train = np.array(y_train)
        groups_train = np.array(groups_train)
        
        return X_train, y_train, groups_train

In [61]:
importer = Import()

In [62]:
X, y, groups = importer.import_from_file('train_groups.csv')

In [63]:
X.shape

(11690, 125)

In [42]:
data

{1: [(15731, 'ВАЗ 21213 | Замена подшипников ступицы | Нива', 0),
  (14829,
   'Ваз 2107 оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
   0),
  (15764,
   'Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена, замена, тюнинг.',
   0),
  (17669, 'Классика 21010 - 21074', 0),
  (14852, 'Ступица Нива — замена подшипника своими руками', 0),
  (15458, 'ВАЗ 2110', 0),
  (14899, 'Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123', 0),
  (16879, 'Купить Подшипники и ступицы FAG (Страница 23)', 0),
  (16310,
   'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года',
   0),
  (15440,
   'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти',
   0),
  (16242,
   'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
   0),
  (16383, 'Ступицы - OLX.ua - страница 80', 0),
  (15580,
   'маааленькая проблемка — бортжурнал Автокам 2160 ╬ 1994 года на DRIV

In [ ]:
s